# Classification: Prediction of sex from possum body measures (exclude case, site, Pop)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

## Load and explore dataset

In [2]:
possum = pd.read_csv('possum.csv')

In [3]:
print(possum.head())

   case  site  Pop sex  age  hdlngth  skullw  totlngth  taill  footlgth  \
0     1     1  Vic   m  8.0     94.1    60.4      89.0   36.0      74.5   
1     2     1  Vic   f  6.0     92.5    57.6      91.5   36.5      72.5   
2     3     1  Vic   f  6.0     94.0    60.0      95.5   39.0      75.4   
3     4     1  Vic   f  6.0     93.2    57.1      92.0   38.0      76.1   
4     5     1  Vic   f  2.0     91.5    56.3      85.5   36.0      71.0   

   earconch   eye  chest  belly  
0      54.5  15.2   28.0   36.0  
1      51.2  16.0   28.5   33.0  
2      51.9  15.5   30.0   34.0  
3      52.2  15.2   28.0   34.0  
4      53.2  15.1   28.5   33.0  


In [4]:
print(possum.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   case      104 non-null    int64  
 1   site      104 non-null    int64  
 2   Pop       104 non-null    object 
 3   sex       104 non-null    object 
 4   age       102 non-null    float64
 5   hdlngth   104 non-null    float64
 6   skullw    104 non-null    float64
 7   totlngth  104 non-null    float64
 8   taill     104 non-null    float64
 9   footlgth  103 non-null    float64
 10  earconch  104 non-null    float64
 11  eye       104 non-null    float64
 12  chest     104 non-null    float64
 13  belly     104 non-null    float64
dtypes: float64(10), int64(2), object(2)
memory usage: 11.5+ KB
None


In [5]:
print(possum.describe())

             case        site         age     hdlngth      skullw    totlngth  \
count  104.000000  104.000000  102.000000  104.000000  104.000000  104.000000   
mean    52.500000    3.625000    3.833333   92.602885   56.883654   87.088462   
std     30.166206    2.349086    1.909244    3.573349    3.113426    4.310549   
min      1.000000    1.000000    1.000000   82.500000   50.000000   75.000000   
25%     26.750000    1.000000    2.250000   90.675000   54.975000   84.000000   
50%     52.500000    3.000000    3.000000   92.800000   56.350000   88.000000   
75%     78.250000    6.000000    5.000000   94.725000   58.100000   90.000000   
max    104.000000    7.000000    9.000000  103.100000   68.600000   96.500000   

            taill    footlgth    earconch         eye       chest       belly  
count  104.000000  103.000000  104.000000  104.000000  104.000000  104.000000  
mean    37.009615   68.459223   48.130769   15.046154   27.000000   32.586538  
std      1.959518    4.395306 

In [6]:
print(possum.value_counts(possum['site']))

site
1    33
7    18
2    13
5    13
6    13
3     7
4     7
dtype: int64


## Use all features except case, site, Pop

In [7]:
possum = possum.drop(['case', 'site', 'Pop'], axis=1)

In [8]:
possum.head()

,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,f,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,f,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


## Handle missing data

### Count missing values

In [9]:
print(possum.isna().sum())

sex         0
age         2
hdlngth     0
skullw      0
totlngth    0
taill       0
footlgth    1
earconch    0
eye         0
chest       0
belly       0
dtype: int64


### Calculate threshold for dropping observations with missing values

In [10]:
treshold = len(possum) * 0.05
print(treshold)

5.2


### Drop all rows with missing values for columns below treshold

We have missing values in age and footlgth below treshold - drop missing value in these columns.

In [11]:
possum.dropna(subset=['footlgth', 'age'], inplace=True)
print(possum.isna().sum())

sex         0
age         0
hdlngth     0
skullw      0
totlngth    0
taill       0
footlgth    0
earconch    0
eye         0
chest       0
belly       0
dtype: int64


## Get X and y from dataframe

In [12]:
X = possum.iloc[:, 1:-1].values
y = possum.iloc[:, 0].values

In [13]:
print(X)
print(y)

[[  8.   94.1  60.4  89.   36.   74.5  54.5  15.2  28. ]
 [  6.   92.5  57.6  91.5  36.5  72.5  51.2  16.   28.5]
 [  6.   94.   60.   95.5  39.   75.4  51.9  15.5  30. ]
 [  6.   93.2  57.1  92.   38.   76.1  52.2  15.2  28. ]
 [  2.   91.5  56.3  85.5  36.   71.   53.2  15.1  28.5]
 [  1.   93.1  54.8  90.5  35.5  73.2  53.6  14.2  30. ]
 [  2.   95.3  58.2  89.5  36.   71.5  52.   14.2  30. ]
 [  6.   94.8  57.6  91.   37.   72.7  53.9  14.5  29. ]
 [  9.   93.4  56.3  91.5  37.   72.4  52.9  15.5  28. ]
 [  6.   91.8  58.   89.5  37.5  70.9  53.4  14.4  27.5]
 [  9.   93.3  57.2  89.5  39.   77.2  51.3  14.9  31. ]
 [  5.   94.9  55.6  92.   35.5  71.7  51.   15.3  28. ]
 [  5.   95.1  59.9  89.5  36.   71.   49.8  15.8  27. ]
 [  3.   95.4  57.6  91.5  36.   74.3  53.7  15.1  28. ]
 [  5.   92.9  57.6  85.5  34.   69.7  51.8  15.7  28. ]
 [  4.   91.6  56.   86.   34.5  73.   51.4  14.4  28. ]
 [  1.   94.7  67.7  89.5  36.5  73.2  53.2  14.7  29. ]
 [  2.   93.5  55.7  90.   36. 

## Encode the dependent variable

In [14]:
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

[1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 0
 1 0 0 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0]


## Split data into training and test set

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[  3.   90.4  55.8  86.   36.5  63.2  44.2  15.7  26.5]
 [  6.   93.2  57.1  92.   38.   76.1  52.2  15.2  28. ]
 [  6.   93.8  58.1  89.   38.   66.2  45.6  16.9  26. ]
 [  4.   95.4  59.2  85.   37.   69.   45.   15.9  29.5]
 [  6.   97.6  61.   93.5  40.   67.9  44.3  15.8  28.5]
 [  2.   95.3  58.2  89.5  36.   71.5  52.   14.2  30. ]
 [  3.   93.6  59.9  89.   40.   67.6  46.   14.8  28.5]
 [  3.   88.2  53.2  86.5  38.5  60.3  43.7  13.6  26. ]
 [  1.   86.7  52.6  84.   38.   62.3  44.8  15.   23.5]
 [  3.   96.9  56.5  89.5  38.5  63.   45.1  17.1  25.5]
 [  3.   88.4  54.6  80.5  36.   62.6  43.6  16.3  25. ]
 [  4.   91.6  56.6  88.5  37.5  64.5  45.4  14.9  27. ]
 [  1.   85.9  52.4  80.5  35.   62.   42.4  14.1  25.5]
 [  2.  103.1  63.2  92.5  38.   72.5  44.9  16.4  30.5]
 [  4.   93.8  56.8  87.   34.5  73.2  53.   15.3  27. ]
 [  5.   94.4  55.4  90.5  35.   73.4  53.9  15.2  28. ]
 [  2.   91.3  57.7  88.   39.   63.1  47.   14.4  26. ]
 [  4.   93.3  57.6  85.   36.5

## Scale data

In [17]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
print(X_train)
print(X_test)

[[-4.85685494e-01 -6.61160112e-01 -3.84831119e-01 -2.42562372e-01
  -2.55433628e-01 -1.14472877e+00 -9.54291658e-01  5.19891382e-01
  -2.93464163e-01]
 [ 1.11108873e+00  1.37652537e-01  1.33594223e-01  1.23685235e+00
   4.74376737e-01  1.78450527e+00  1.04030880e+00  8.57440902e-02
   5.65455339e-01]
 [ 1.11108873e+00  3.08826676e-01  5.32382947e-01  4.97144989e-01
   4.74376737e-01 -4.63511550e-01 -6.05236577e-01  1.56184488e+00
  -5.79770664e-01]
 [ 4.65725816e-02  7.65291046e-01  9.71050544e-01 -4.89131493e-01
  -1.21635061e-02  1.72291188e-01 -7.54831612e-01  6.93550299e-01
   1.42437484e+00]
 [ 1.11108873e+00  1.39292956e+00  1.68887025e+00  1.60670603e+00
   1.44745722e+00 -7.74884587e-02 -9.29359152e-01  6.06720840e-01
   8.51761840e-01]
 [-1.01794357e+00  7.36762023e-01  5.72261819e-01  6.20429550e-01
  -4.98703750e-01  7.39972204e-01  9.90443791e-01 -7.82550494e-01
   1.71068134e+00]
 [-4.85685494e-01  2.51768629e-01  1.25020265e+00  4.97144989e-01
   1.44745722e+00 -1.4561018

## Create Logistic Regression model on training data

In [19]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

## Predict Test set

In [20]:
y_pred = classifier.predict(X_test)

In [21]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 0]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [1 0]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [0 1]]


## Print Confusion Matrix

In [22]:
print(confusion_matrix(y_test, y_pred))

[[5 3]
 [6 7]]


In [23]:
print(accuracy_score(y_test, y_pred) * 100)

57.14285714285714


## Computing the accuracy with k-Fold Cross Validation

In [24]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 61.25 %
Standard Deviation: 17.18 %


## => Conclusion: not a good model. 